In [ ]:
### 0. import
import psycopg2
import numpy  as np
import pandas as pd
from shapely.wkb import loads
from datetime import datetime

# display option (float 자리수)
pd.options.display.float_format = lambda x: f'{x:.4f}'


### 1. 데이터 가져오기
# AWS DW Connect
aws_conn = psycopg2.connect(
    host = 'redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
    port = 5439,
    dbname = 'dev',
    user = 'awsuser',
    password = 'cremaoAdmin1234qwer!!'
)
aws_cursor = aws_conn.cursor()

# pnu별 상업용 부동산 면적당 업종별 추정 매출액
sql = '''
with date as (
    select '202209' as base_ym
)
select substring(A.bd_mgt_sn, 1, 19) as pnu, B.emd_cd, A.ftc_cate2_cd, round(A.pred_slng_amt/C.tot_area, 2) as sales_per_area
from ( -- 건물당 매출액, 업종코드 
	select *
	from m1.kt_bldg_sales 
	where base_ym = (select base_ym from date)
	  and pred_slng_amt notnull 
	  and pred_slng_amt > 0
	) A
join m1.pnu B -- 실제 pnu여부 확인
	on substring(A.bd_mgt_sn, 1, 19) = B.pnu
join (-- 연면적 
	select sigungu_cd||bjdong_cd||case when plat_gb_cd = '0' then '1' else plat_gb_cd end||bun||ji as pnu, sum(tot_area) as tot_area
	from m1.bld_rgst
	where base_ym = (select base_ym from date)
	  and tot_area notnull
	  and tot_area > 0
	group by 1
) C
	on substring(A.bd_mgt_sn, 1, 19) = C.pnu
'''
aws_cursor.execute(sql)
aws_conn.commit()
rows = aws_cursor.fetchall()

# polygon 변환
rows1 = []
for k in range(len(rows)):
    rows1.append((rows[k][:4]) + (loads(rows[k][4], hex=True).wkt,))

# 데이터프레임화
df = pd.DataFrame(rows1)
df.columns = ['pnu','emd_cd','industry_cd','sales_per_area','poly']

# 작업일(업데이트일) 및 작업자 추가
df['create_at'] = datetime.today().strftime("%Y-%m-%d")
df['update_at'] = datetime.today().strftime("%Y-%m-%d")
df['work_user'] = 'du.Park'

In [ ]:
### 2. DB INSERT
import datetime

table = 'm2.datadam_sales_per_area'

# execute_mogrify
def execute_mogrify(conn, df, table, val):
    # Create a list of tuples from the dataframe values
    tuples = [tuple(x) for x in df.to_numpy()]
    # Comma-separated dataframe columns
    cols = ','.join(list(df.columns))
    # SQL query to execute
    cursor = conn.cursor()
    values = [cursor.mogrify(val, tup).decode('utf8') for tup in
              tuples]
    query = "INSERT INTO %s(%s) VALUES " % (table, cols) + ",".join(values)

    try:
        cursor.execute(query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    cursor.close()

# values 설정
val = "(" + ("%s," * len(df.columns))[:-4] + "ST_GeomFromText(%s, 4326, 'axis-order=long-lat'))"

# insert
for j in range(0, len(df), 10000):

    # DB Connect
    conn = psycopg2.connect(
        host='redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
        port=5439,
        dbname='dev',
        user='awsuser',
        password='cremaoAdmin1234qwer!!'
    )

    # 10000개 단위로 나누기
    tmp = df[j:j + 10000]

    # insert
    execute_mogrify(conn, tmp, table, val)
    print(datetime.datetime.now(), ' : ', j)
    tmp = pd.DataFrame()

# row count
print(len(df))